### STEP 1: DATA EXTRACTION:

In [ ]:
# Import python libraries

import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 35)
pd.set_option('display.max_columns', 35)

In [ ]:
# Import data from SQL SERVER database

from sqlalchemy import create_engine, MetaData, Table, select, text
import pandas as pd

# Connection string using Windows Authentication (no username/password needed)
# Replace 'your_server_name' and 'your_database_name' with actual values
server_name = r"DEBBY\MSSQLSERVER06"  # e.g., "localhost" or "SERVER01\SQLEXPRESS"
database_name = "datawarehouse"   # your database name

# Create connection string for Windows Authentication
connection_string = f"mssql+pyodbc://{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Alternative connection string format
# connection_string = f"mssql+pyodbc://@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"

# Create engine
engine = create_engine(connection_string)

# Method 1: Using pandas to execute SELECT * (Recommended for data analysis)
try:
    # Simple SELECT * from table
    query = "SELECT * from [datawarehouse].[dbo].[fact_unified_covid]"
    covid_df = pd.read_sql(query, engine)
    print(f"Retrieved {len(covid_df)} rows from table")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# 35 unique PHUs. Investigated 9999 which is unknown PHU

covid_df["PHU_ID"].value_counts(dropna=False)

### STEP 2: DATA CLEANING AND STANDARDIZATION

• Handle missing values using imputation and removal as appropriate. 
• Remove duplicates and check for consistency in identifiers (PHU_ID, Agegroup, Date). 
• Detect and treat outliers in case counts and vaccination data. 
• Standardize variable names and formats across datasets (snake_case format preferred). 
• Validate merged dataset ensuring PHU_ID, Date, and Agegroup align correctly.

#### 1. Standardize variable names and formats across datasets (snake_case format preferred)

In [ ]:
#####  Standardize variable names and formats across datasets (snake_case format preferred)

import pandas as pd
import re

def standardize_column_names(df):
   """
   Convert DataFrame column names to snake_case format
   """
   covid_df_copy = covid_df.copy()
   old_to_new = {}
   
   for col in covid_df_copy.columns:
       # Convert to snake_case
       new_col = col.strip()  # Remove leading/trailing spaces
       new_col = re.sub(r'[^\w\s]', '', new_col)  # Remove special characters except underscore
       new_col = re.sub(r'\s+', '_', new_col)     # Replace spaces with underscore
       new_col = re.sub(r'_{2,}', '_', new_col)   # Replace multiple underscores with single
       new_col = new_col.lower()                  # Convert to lowercase
       new_col = new_col.strip('_')               # Remove leading/trailing underscores
       
       old_to_new[col] = new_col
   
   # Rename columns
   covid_df_copy = covid_df_copy.rename(columns=old_to_new)
   
   # Print changes
   print("Column name changes:")
   for old, new in old_to_new.items():
       if old != new:
           print(f"  '{old}' → '{new}'")
   
   return covid_df_copy

# Apply to your DataFrame
covid_df = standardize_column_names(covid_df)

print(f"\n✅ Standardized {len(covid_df.columns)} column names to snake_case format")
print(f"New columns: {list(covid_df.columns)}")

In [ ]:
# Standardize Dates Data Types

covid_df['date'] = pd.to_datetime(covid_df['date'], errors='coerce')
covid_df['date'] = pd.to_datetime(covid_df['date'], errors='coerce')
covid_df['date'] = pd.to_datetime(covid_df['date'], errors='coerce')
covid_df['case_reported_date'] = pd.to_datetime(covid_df['case_reported_date'], errors='coerce')
covid_df['test_reported_date'] = pd.to_datetime(covid_df['test_reported_date'], errors='coerce')
covid_df['specimen_date'] = pd.to_datetime(covid_df['specimen_date'], errors='coerce')

In [ ]:
# Standardize Numerical Data Types

covid_df['active_cases'] = pd.to_numeric(covid_df['active_cases'], errors='coerce').astype('Int64')
covid_df['resolved_cases'] = pd.to_numeric(covid_df['resolved_cases'], errors='coerce').astype('Int64')
covid_df['deaths'] = pd.to_numeric(covid_df['deaths'], errors='coerce').astype('Int64')
covid_df['positive_cases_agegroup'] = pd.to_numeric(covid_df['positive_cases_agegroup'], errors='coerce').astype('Int64')
covid_df['positive_cases_total'] = pd.to_numeric(covid_df['positive_cases_total'], errors='coerce').astype('Int64')
covid_df['cases_by_case_reported_agegrp'] = pd.to_numeric(covid_df['cases_by_case_reported_agegrp'], errors='coerce').astype('Int64')
covid_df['cases_by_test_reported_agegrp'] = pd.to_numeric(covid_df['cases_by_test_reported_agegrp'], errors='coerce').astype('Int64')
covid_df['cases_by_specimen_agegrp'] = pd.to_numeric(covid_df['cases_by_specimen_agegrp'], errors='coerce').astype('Int64')
covid_df['cases_by_case_reported_total'] = pd.to_numeric(covid_df['cases_by_case_reported_total'], errors='coerce').astype('Int64')
covid_df['cases_by_test_reported_total'] = pd.to_numeric(covid_df['cases_by_test_reported_total'], errors='coerce').astype('Int64')
covid_df['cases_by_specimen_total'] = pd.to_numeric(covid_df['cases_by_specimen_total'], errors='coerce').astype('Int64')

#### 2. Remove duplicates and check for consistency in identifiers (PHU_ID, Agegroup, Date)

In [ ]:
# CHECK FOR DUPLICATES: NO DUPLICATES

covid_df[covid_df.duplicated()]

#### 3. Handle missing values using imputation or removal as appropriate

In [ ]:
#  CHECK MISSING VALUES: Counts

covid_df.isna().sum()

In [ ]:
#  CHECK MISSING VALUES: Percentages

(covid_df
 .isna()
 .mean() 
 .mul(100)
 .pipe(lambda ser: ser[ser > 0])
)

#### 4. Detect and treat outliers in case counts and vaccination data

In [ ]:
# DETECTING AND HANDLING OUTLIERS USING IQR METHOD:


import pandas as pd
import numpy as np
# Work on a copy to avoid modifying the original DataFrame
covid_df_copy= covid_df.copy()
# Outlier handling – cap using IQR
for col in covid_df_copy.select_dtypes(include=[np.number]).columns:
   Q1 = covid_df[col].quantile(0.25)
   Q3 = covid_df[col].quantile(0.75)
   IQR = Q3 - Q1
   lower_bound = Q1 - 1.5 * IQR
   upper_bound = Q3 + 1.5 * IQR
   
   # Convert nullable Int64 columns to float64 to handle negative values
   if covid_df_copy[col].dtype == 'Int64':
       covid_df_copy[col] = covid_df_copy[col].astype('float64')
   
   covid_df_copy.loc[covid_df_copy[col] < lower_bound, col] = lower_bound
   covid_df_copy.loc[covid_df_copy[col] > upper_bound, col] = upper_bound

print("✅ Missing values imputed and outliers capped on a safe copy.")